# Module 05 — Semiconductor Zoo
# Notebook 01: Power and Regulation
## TL431, LDOs, IGBTs, Peltier Coolers

---

Digital circuits, sensors, and analog amplifiers all need stable supply voltages.
A power supply's raw output is rarely clean enough — we need **voltage regulators**
to deliver a precise, stable voltage regardless of load changes or input fluctuations.
This notebook explores the semiconductor devices that make that possible.

---
## Concept — Why Voltage Regulation Matters

Consider what happens without regulation:

- A battery's voltage drops as it discharges (4.2 V to 3.0 V for LiPo)
- A wall adapter's output sags under load and has AC ripple
- Different parts of a system need different voltages (3.3 V, 5 V, 12 V)

A **voltage regulator** takes an unregulated input and outputs a stable, precise voltage.

Two fundamental approaches:

| Type | How it works | Efficiency | Noise | Complexity |
|------|-------------|-----------|-------|------------|
| **Linear** (LDO) | Dissipates excess voltage as heat | Low–Medium | Very low | Simple |
| **Switching** (Buck/Boost) | Rapidly switches and filters | High | Higher | Complex |

---
## Concept — The TL431 Shunt Voltage Regulator

The **TL431** is one of the most widely used ICs in power supplies. It is a
**programmable precision shunt regulator** — essentially a smart Zener diode
whose breakdown voltage you set with two resistors.

### How it works:

```
  Cathode (K) ──────┬──── V_out
                    │
              ┌─────┤
              │ TL431│
              │     ├──── Reference (Ref)
              └─────┤
                    │
  Anode (A) ────────┴──── GND
```

The TL431 has an internal **2.495 V reference**. It compares the voltage on the
**Ref** pin to this reference:

- If V_ref > 2.495 V: the device conducts more (pulls cathode toward anode)
- If V_ref < 2.495 V: the device conducts less

By connecting a **resistor divider** from V_out to the Ref pin, you set the output
voltage:

$$V_{out} = V_{ref} \times \left(1 + \frac{R_1}{R_2}\right) = 2.495 \times \left(1 + \frac{R_1}{R_2}\right)$$

where $R_1$ is from V_out to Ref, and $R_2$ is from Ref to GND.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# --- TL431 Voltage Divider Calculator ---

V_ref = 2.495  # TL431 internal reference voltage

# Common E24 resistor values (kohm)
e24 = [1.0, 1.1, 1.2, 1.3, 1.5, 1.6, 1.8, 2.0, 2.2, 2.4, 2.7, 3.0,
       3.3, 3.6, 3.9, 4.3, 4.7, 5.1, 5.6, 6.2, 6.8, 7.5, 8.2, 9.1, 10.0]

def tl431_vout(R1_kohm, R2_kohm):
    """Calculate TL431 output voltage from R1 (Vout-to-Ref) and R2 (Ref-to-GND)."""
    return V_ref * (1 + R1_kohm / R2_kohm)

# Example calculations
target_voltages = [3.3, 5.0, 9.0, 12.0]

print('TL431 Resistor Divider Calculator')
print('=' * 60)
print(f'Internal Reference: {V_ref} V')
print()

for V_target in target_voltages:
    # Find best E24 resistor pair
    best_err = 999
    best_R1, best_R2 = 0, 0
    for R2 in e24:
        # R1/R2 = (Vout/Vref) - 1
        R1_ideal = R2 * (V_target / V_ref - 1)
        # Find closest E24
        for mult in [0.1, 1.0, 10.0, 100.0]:
            for r in e24:
                R1_try = r * mult
                V_actual = tl431_vout(R1_try, R2)
                err = abs(V_actual - V_target)
                if err < best_err:
                    best_err = err
                    best_R1 = R1_try
                    best_R2 = R2
                    best_V = V_actual

    print(f'Target: {V_target:.1f} V')
    print(f'  R1 = {best_R1:.1f} kohm, R2 = {best_R2:.1f} kohm')
    print(f'  Actual Vout = {best_V:.3f} V (error: {best_err*1000:.1f} mV)')
    print()

---
## Concept — Linear Regulators (LDOs)

A **Low Dropout (LDO) regulator** is a linear voltage regulator that can operate
with a very small difference between input and output voltage.

### How it works:
1. A **pass transistor** (usually PMOS for LDO) sits between Vin and Vout
2. An **error amplifier** compares Vout (via a divider) to an internal reference
3. The error amplifier adjusts the pass transistor's gate to maintain constant Vout

```
  Vin ──┬──[Pass Transistor]──┬── Vout
        │         │ gate      │
        │    ┌────┘           │
        │    │ Error          │
        │    │ Amp  ┌─[R1]───┤
        │    │  +───┘        │
        │    │  -───Vref     ├─[R2]──GND
        │    └───────┘       │
       Cin                  Cout
        │                    │
       GND                  GND
```

### Key parameters:
- **Dropout voltage**: Minimum $V_{in} - V_{out}$ for regulation (200 mV for good LDOs)
- **Quiescent current**: How much current the regulator itself consumes
- **PSRR**: Power Supply Rejection Ratio — how well it filters input noise
- **Load regulation**: How much Vout changes with load current
- **Line regulation**: How much Vout changes with input voltage

### The Heat Problem

All excess voltage is dissipated as heat:

$$P_{dissipated} = (V_{in} - V_{out}) \times I_{out}$$

Example: Regulating 12 V down to 3.3 V at 500 mA:

$$P = (12 - 3.3) \times 0.5 = 4.35 \text{ W}$$

That is a lot of heat for a small IC! This is why LDOs are best when the voltage
difference is small.

In [ ]:
# --- LDO Power Dissipation vs Input/Output Voltage Difference ---

I_out_values = [0.1, 0.25, 0.5, 1.0]  # Amps
V_dropout = np.linspace(0, 10, 200)    # Vin - Vout

fig, ax = plt.subplots(figsize=(10, 6))

colors = ['#2196F3', '#4CAF50', '#FF9800', '#F44336']
for I_out, color in zip(I_out_values, colors):
    P = V_dropout * I_out
    ax.plot(V_dropout, P, linewidth=2, color=color, label=f'$I_{{out}}$ = {I_out} A')

# Mark typical scenarios
scenarios = [
    (0.5, 0.5, '5V -> 3.3V\n0.5A', '3.3V LDO'),
    (8.7, 0.5, '12V -> 3.3V\n0.5A', 'Ouch!'),
    (1.7, 1.0, '5V -> 3.3V\n1A', 'Warm'),
]
for vd, iout, label, _ in scenarios:
    p = vd * iout
    ax.plot(vd, p, 'ko', markersize=8)
    ax.annotate(label, (vd, p), textcoords='offset points',
                xytext=(10, 10), fontsize=8,
                arrowprops=dict(arrowstyle='->', color='black'))

# SOT-223 thermal limit (approx)
ax.axhline(y=1.5, color='red', linestyle='--', alpha=0.5,
           label='SOT-223 thermal limit (~1.5 W)')

ax.set_xlabel('Voltage Drop: $V_{in} - V_{out}$ (V)', fontsize=12)
ax.set_ylabel('Power Dissipated (W)', fontsize=12)
ax.set_title('LDO Power Dissipation — All Wasted as Heat', fontsize=14)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 10)
ax.set_ylim(0, 10)
plt.tight_layout()
plt.show()

In [ ]:
# --- Compare MOSFET vs IGBT vs BJT conduction losses ---

I = np.linspace(0, 50, 200)  # Amps

# MOSFET: V_ds = I * R_ds(on) — resistive, increases with current
R_ds_on = 0.05  # 50 mOhm for a typical power MOSFET
P_mosfet = I**2 * R_ds_on

# IGBT: V_ce(sat) ~ constant + small resistive component
V_ce_sat = 1.5   # typical saturation voltage
R_igbt = 0.02    # small resistive component
P_igbt = I * V_ce_sat + I**2 * R_igbt

# BJT: V_ce(sat) ~ constant
V_ce_bjt = 0.7
R_bjt = 0.05
P_bjt = I * V_ce_bjt + I**2 * R_bjt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(I, P_mosfet, 'b-', linewidth=2, label=f'MOSFET ($R_{{ds(on)}}$ = {R_ds_on*1000:.0f} m$\Omega$)')
ax1.plot(I, P_igbt, 'r-', linewidth=2, label=f'IGBT ($V_{{ce(sat)}}$ = {V_ce_sat} V)')
ax1.plot(I, P_bjt, 'g-', linewidth=2, label=f'BJT ($V_{{ce(sat)}}$ = {V_ce_bjt} V)')

# Crossover point MOSFET vs IGBT
cross_idx = np.argmin(np.abs(P_mosfet - P_igbt))
ax1.axvline(x=I[cross_idx], color='gray', linestyle='--', alpha=0.5)
ax1.annotate(f'Crossover ~{I[cross_idx]:.0f} A', (I[cross_idx], P_mosfet[cross_idx]),
             textcoords='offset points', xytext=(10, 20), fontsize=9)

ax1.set_xlabel('Current (A)', fontsize=12)
ax1.set_ylabel('Conduction Loss (W)', fontsize=12)
ax1.set_title('Conduction Losses: MOSFET vs IGBT vs BJT', fontsize=13)
ax1.legend(fontsize=9)
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, 50)

# Summary comparison table
comparison = [
    ['Feature', 'MOSFET', 'IGBT', 'BJT'],
    ['Drive', 'Voltage', 'Voltage', 'Current'],
    ['Speed', 'Fast', 'Medium', 'Slow'],
    ['V_drop', 'I×R_ds', '~1-3V', '~0.2-0.7V'],
    ['Best for', '<30A', '>30A, HV', 'Low power'],
    ['Example', 'DC-DC conv', 'EV motor', 'Audio amp'],
]

ax2.axis('off')
table = ax2.table(cellText=comparison[1:], colLabels=comparison[0],
                  cellLoc='center', loc='center')
table.auto_set_font_size(False)
table.set_fontsize(11)
table.scale(1.0, 1.8)
for (row, col), cell in table.get_celld().items():
    if row == 0:
        cell.set_facecolor('#4472C4')
        cell.set_text_props(color='white', fontweight='bold')
    elif row % 2 == 0:
        cell.set_facecolor('#D9E2F3')
ax2.set_title('Device Comparison', fontsize=13, pad=20)

plt.tight_layout()
plt.show()

---
## The Material Science Why — IGBTs

The **IGBT (Insulated Gate Bipolar Transistor)** combines the best of both worlds:

- **MOSFET gate**: High input impedance, voltage-controlled, easy to drive
- **BJT output**: Low saturation voltage at high currents (lower conduction losses)

Structurally, an IGBT is a MOSFET with an extra P+ layer at the drain (called the
collector). This forms a PNP transistor that takes over current conduction when
the device is on.

```
  MOSFET structure:     IGBT structure:
  Gate                  Gate
   │                     │
  ═══                   ═══
  N+ Source             N+ Emitter
  P body                P body
  N- drift              N- drift
  N+ Drain              P+ Collector   <-- Extra layer!
```

The extra P+ layer injects minority carriers (holes) into the drift region,
reducing its effective resistance dramatically. This is called **conductivity
modulation** — and it is why IGBTs can handle hundreds of amps at 1200 V in
electric vehicle inverters.

The trade-off: those injected carriers must be removed when turning off (tail
current), making IGBTs slower than MOSFETs.

---
## The Material Science Why — Peltier Coolers

A **Peltier cooler** (thermoelectric cooler, TEC) uses the **Peltier effect**:
when current flows through a junction of two different semiconductor materials,
heat is absorbed on one side and released on the other.

Inside a TEC module:
- Many pairs of N-type and P-type bismuth telluride (Bi2Te3) elements
- Connected electrically in series, thermally in parallel
- Current flowing N -> P absorbs heat (cold side)
- Current flowing P -> N releases heat (hot side)

```
  Cold side (heat absorbed)
  ═══════════════════════════
  │ N │ P │ N │ P │ N │ P │    Semiconductor pellets
  ═══════════════════════════
  Hot side (heat released)

  Current path: serpentine through all N-P pairs
```

The **Seebeck effect** is the reverse: a temperature difference across a
junction generates a voltage. This is how thermocouples work.

Applications:
- CPU/GPU spot cooling
- Portable coolers (12V car fridges)
- Laser diode temperature stabilization
- Wine coolers
- Dew point sensors

In [ ]:
# --- Interactive: TL431 Circuit ---

import ipywidgets as widgets
from IPython.display import display

def plot_tl431(R1_kohm=10.0, R2_kohm=10.0, Vin=12.0):
    """TL431 shunt regulator output voltage calculator with visualization."""
    V_ref = 2.495
    V_out = V_ref * (1 + R1_kohm / R2_kohm)

    # Ensure Vout does not exceed Vin (TL431 cannot boost)
    if V_out > Vin - 0.5:
        V_out_actual = Vin - 0.5  # won't regulate
        regulating = False
    else:
        V_out_actual = V_out
        regulating = True

    # Sweep R1 to show how Vout changes
    R1_sweep = np.linspace(0.1, 50, 300)
    Vout_sweep = V_ref * (1 + R1_sweep / R2_kohm)
    Vout_sweep = np.clip(Vout_sweep, 0, Vin)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    # Left: Vout vs R1
    ax1.plot(R1_sweep, Vout_sweep, 'b-', linewidth=2)
    ax1.plot(R1_kohm, V_out_actual, 'ro', markersize=12,
             label=f'R1={R1_kohm:.1f}k: Vout={V_out_actual:.2f}V')
    ax1.axhline(y=V_ref, color='green', linestyle='--', alpha=0.5,
                label=f'Min Vout = {V_ref}V (R1=0)')
    ax1.axhline(y=Vin, color='red', linestyle='--', alpha=0.5,
                label=f'Vin = {Vin}V (max)')
    ax1.set_xlabel('R1 (kohm)', fontsize=12)
    ax1.set_ylabel('Output Voltage (V)', fontsize=12)
    ax1.set_title(f'TL431 Output vs R1 (R2={R2_kohm:.1f}k)', fontsize=13)
    ax1.legend(fontsize=9)
    ax1.grid(True, alpha=0.3)
    ax1.set_xlim(0, 50)

    # Right: circuit status text
    ax2.axis('off')
    status = 'REGULATING' if regulating else 'NOT REGULATING (Vout > Vin)'
    status_color = '#2E7D32' if regulating else '#C62828'

    info = (f"TL431 Shunt Regulator\n"
            f"{'='*30}\n\n"
            f"Vin:    {Vin:.1f} V\n"
            f"R1:     {R1_kohm:.1f} kohm\n"
            f"R2:     {R2_kohm:.1f} kohm\n"
            f"Ratio:  R1/R2 = {R1_kohm/R2_kohm:.3f}\n\n"
            f"Vout = 2.495 x (1 + {R1_kohm:.1f}/{R2_kohm:.1f})\n"
            f"Vout = {V_out:.3f} V\n\n"
            f"Status: {status}")

    ax2.text(0.1, 0.5, info, fontsize=13, fontfamily='monospace',
             verticalalignment='center', transform=ax2.transAxes,
             bbox=dict(boxstyle='round', facecolor='lightyellow', edgecolor='gray'))

    plt.tight_layout()
    plt.show()

widgets.interact(
    plot_tl431,
    R1_kohm=widgets.FloatSlider(value=10.0, min=0.1, max=50.0, step=0.1,
                                 description='R1 (kohm)'),
    R2_kohm=widgets.FloatSlider(value=10.0, min=1.0, max=50.0, step=0.1,
                                 description='R2 (kohm)'),
    Vin=widgets.FloatSlider(value=12.0, min=3.0, max=30.0, step=0.5,
                             description='Vin (V)')
);

---
## Experiment — TL431 Adjustable Voltage Regulator

### Equipment
- Breadboard and jumper wires
- TL431 (TO-92 package)
- Assorted resistors (1k, 2.2k, 4.7k, 10k, 22k, 47k)
- 100 ohm power resistor (load)
- Bench power supply (set to 12V)
- Klein MM300 multimeter

### Experiment 1: Build the TL431 Regulator

```
  Vin (12V)
   │
   ┌───┐
   │ R_s│  470 ohm (series resistor — provides current path)
   └─┬─┘
     │
     ├───────────────────── V_out (measure here)
     │
     │    TL431 (TO-92, flat side facing you)
     │    ┌─────────┐
     ├────┤ K   Ref ├────┬───── voltage divider midpoint
     │    │    A    │    │
     │    └────┬────┘    │
     │         │         │
     │        GND    ┌───┴───┐
     │               │  R1   │  (Vout to Ref)
     │               │ 10k   │
     │               └───┬───┘
     │                   │
     ├──── R_load ───────┤
     │    (100 ohm)      │
     │               ┌───┴───┐
     │               │  R2   │  (Ref to GND)
     │               │ 10k   │
     │               └───┬───┘
     │                   │
    GND                 GND
```

**Pin identification (TL431 TO-92, flat side facing you):**
- Left pin: Cathode (K)
- Center pin: Reference (Ref)
- Right pin: Anode (A) — connect to GND

**Procedure:**
1. Wire the circuit as shown with R1 = R2 = 10k
2. Expected Vout = 2.495 x (1 + 10/10) = **4.99 V**
3. Measure Vout with multimeter
4. Change R1 to 22k: expected Vout = 2.495 x (1 + 22/10) = **7.99 V**
5. Change R1 to 4.7k: expected Vout = 2.495 x (1 + 4.7/10) = **3.67 V**

### Experiment 2: Line Regulation — Vary Input Voltage

With R1 = R2 = 10k (target 5V output):

1. Set bench supply to 8V, measure Vout
2. Increase to 10V, measure Vout
3. Increase to 12V, measure Vout
4. Increase to 15V, measure Vout
5. Increase to 20V, measure Vout

**Expected:** Vout should remain very close to 5.0V across the range.
The TL431 achieves excellent line regulation because its internal
amplifier adjusts the shunt current to maintain V_ref on the Ref pin.

### Experiment 3: Load Regulation

With Vin = 12V, R1 = R2 = 10k:

1. No load resistor: measure Vout
2. 1k load: measure Vout
3. 470 ohm load: measure Vout
4. 100 ohm load: measure Vout (check if R_s can supply enough current)

### Experiment 4: Compare to Zener (Module 02)

Recall from Module 02 how a 5.1V Zener diode regulates voltage.
Compare the TL431 output stability to the Zener under the same line
and load changes. The TL431 should be significantly more stable.

---
## Simulation

- [TL431 Shunt Regulator on Falstad](https://www.falstad.com/circuit/) — 
  Build a shunt regulator: voltage source + series resistor + Zener model
  (the TL431 is not directly available but behaves like a precision Zener)
- [LDO concept](https://www.falstad.com/circuit/e-opamp-follower.html) —
  An op-amp voltage follower shows the basic LDO concept: the output tracks
  a reference voltage

---
## Datasheet Connection

### TL431 Datasheet Key Parameters

| Parameter | Typical | What it means |
|-----------|---------|---------------|
| Reference voltage | 2.495 V | Internal comparison voltage |
| Reference tolerance | +/- 0.4% | How accurate the 2.495V is |
| Cathode voltage range | 2.5–36 V | Output voltage range |
| Cathode current range | 1–100 mA | Min/max shunt current |
| Dynamic impedance | 0.22 ohm | How stiff the regulation is |
| Temperature coefficient | 5 mV/C typical | Drift with temperature |

### LDO Datasheet Key Parameters

| Parameter | Example (AMS1117-3.3) | What it means |
|-----------|----------------------|---------------|
| Output voltage | 3.3 V | Fixed output |
| Dropout voltage | 1.1 V @ 1A | Min Vin = 4.4V for 3.3V out |
| Max output current | 1 A | Don't exceed this |
| Line regulation | 0.2%/V | Vout change per Vin change |
| Load regulation | 0.4% | Vout change from no-load to full-load |
| Quiescent current | 5 mA | Current consumed by regulator itself |

### IGBT Datasheet

| Parameter | What to look for |
|-----------|------------------|
| V_CE(sat) | Collector-emitter saturation voltage (conduction loss) |
| V_GE(th) | Gate threshold voltage (how much to drive it) |
| I_C | Maximum continuous collector current |
| E_on, E_off | Switching energy losses (per cycle) |
| t_on, t_off | Switching times |

---
## Checkpoint Questions

1. **You need a TL431 circuit to output 9.0V. If R2 = 10k, what value of R1 do you need?**
   (Show your calculation.)

2. **An LDO regulates 5V to 3.3V at 800 mA. How much power does it dissipate as heat?
   Could you use a SOT-223 package (rated ~1.5W)?**

3. **Why is an LDO a poor choice for regulating 24V down to 3.3V at 1A?
   What type of regulator would you use instead?**

4. **What advantage does an IGBT have over a MOSFET at 600V, 50A?
   What advantage does the MOSFET have at 12V, 5A?**

5. **A Peltier cooler has a "cold side" and a "hot side." What happens
   if you reverse the current direction?**

6. **The TL431 requires a minimum cathode current of 1 mA to regulate.
   If your load draws 50 mA and Vin = 12V with Vout = 5V, what is the
   minimum series resistor value? (Hint: the series resistor must supply
   both the load current AND the minimum TL431 current.)**

In [ ]:
# --- Quick calculation helpers for checkpoint questions ---

# Q1: R1 for 9.0V output
V_target = 9.0
R2 = 10.0  # kohm
V_ref = 2.495
R1_needed = R2 * (V_target / V_ref - 1)
print(f'Q1: R1 = R2 x (Vout/Vref - 1) = {R2} x ({V_target}/{V_ref} - 1) = {R1_needed:.2f} kohm')
print(f'    Nearest standard: 27 kohm -> Vout = {V_ref * (1 + 27/R2):.3f} V')
print()

# Q2: LDO power dissipation
Vin, Vout, Iout = 5.0, 3.3, 0.8
P_ldo = (Vin - Vout) * Iout
print(f'Q2: P = ({Vin} - {Vout}) x {Iout} = {P_ldo:.2f} W')
print(f'    SOT-223 rated ~1.5W: {"YES, safe" if P_ldo < 1.5 else "NO, too hot"}')
print()

# Q6: Minimum series resistor
Vin, Vout = 12.0, 5.0
I_load = 0.050   # 50 mA
I_min_tl431 = 0.001  # 1 mA
I_total = I_load + I_min_tl431
R_s_max = (Vin - Vout) / I_total
print(f'Q6: I_total = {I_load*1000:.0f} mA + {I_min_tl431*1000:.0f} mA = {I_total*1000:.0f} mA')
print(f'    R_s(max) = ({Vin} - {Vout}) / {I_total} = {R_s_max:.1f} ohm')
print(f'    Use a value smaller than {R_s_max:.0f} ohm to ensure enough current')